>[Downloading Model Data](#updateTitle=true&folderId=1Ac9BA6juBSuNY0MybchxlrX93n8x0RCs&scrollTo=m_Vt_gHBOjxd)

>[Configuring TensorFlow](#updateTitle=true&folderId=1Ac9BA6juBSuNY0MybchxlrX93n8x0RCs&scrollTo=5IjH5gusOhy9)

>[Webcam Demo](#updateTitle=true&folderId=1Ac9BA6juBSuNY0MybchxlrX93n8x0RCs&scrollTo=L95kdZi1QDFw)



# Downloading Model Data

Run these steps first to download the TensorFlow model data.


In [0]:
!git clone https://github.com/tensorflow/models
checkpoint_name = 'mobilenet_v2_1.0_224'
!wget https://storage.googleapis.com/mobilenet_v2/checkpoints/{checkpoint_name}.tgz
!tar -xf {checkpoint_name}.tgz
checkpoint = '{0}.ckpt'.format(checkpoint_name)
print('Setup successful!')

fatal: destination path 'models' already exists and is not an empty directory.
--2019-09-19 20:00:34--  https://storage.googleapis.com/mobilenet_v2/checkpoints/mobilenet_v2_1.0_224.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78306834 (75M) [application/x-tar]
Saving to: ‘mobilenet_v2_1.0_224.tgz.1’

mobilenet_v2_1.0_22 100%[===================>]  74.68M   252MB/s    in 0.3s    

2019-09-19 20:00:35 (252 MB/s) - ‘mobilenet_v2_1.0_224.tgz.1’ saved [78306834/78306834]

Setup successful!


# Configuring TensorFlow

These steps start TensorFlow and read the downloaded model data into memory so we can use them.

In [0]:
import tensorflow as tf

# setup path so that mobilenet_v2 can be found.
import sys
sys.path.append('/content/models/research/slim')
from nets.mobilenet import mobilenet_v2

tf.reset_default_graph()

# This section builds a "graph" in TensorFlow to explain how to process the data.
jpeg_input_tensor = tf.placeholder(tf.string, ())  # We will provide a JPEG to TF.

# First, instruct TF to decode the JPEG string into a matrix.
image = tf.image.decode_jpeg(jpeg_input_tensor)

# Resize the image to make it friendlier to mobilenet.
images = tf.expand_dims(image, 0)
images = tf.cast(images, tf.float32) / 128.  - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (224, 224))

# Note: arg_scope is optional for inference.
with tf.contrib.slim.arg_scope(mobilenet_v2.training_scope(is_training=False)):
  logits, endpoints = mobilenet_v2.mobilenet(images)
  
# Restore using exponential moving average since it produces (1.5-2%) higher 
# accuracy
ema = tf.train.ExponentialMovingAverage(0.999)
vars = ema.variables_to_restore()

saver = tf.train.Saver(vars)  

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





# Webcam Demo

This section creates the video input element and connects it to TensorFlow.

In [0]:
# Taken from https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=SucxddsPhOmj
from datasets import imagenet
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var canvas;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    var animationFrameId;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       canvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      console.log('anim' + pendingResolve);
      if (pendingResolve) {
        var result = "";
        if (shutdown) {
          removeDom();
          window.cancelAnimationFrame(animationFrameId);
          animationFrameId = null;
        } else {
          canvas.getContext('2d').drawImage(video, 0, 0);
          result = canvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        shutdown = false;
        lp(result);
      }
      
      animationFrameId = window.requestAnimationFrame(onAnimationFrame);
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
//      video.style.width = '100%';
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Model output:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
      
      div.appendChild(video);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click the video to stop this demo</span>';
      div.appendChild(instruction);
      
      video.srcObject = stream;
      await video.play();
      
      // Resize the output to fit the video element.
      // google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      animationFrameId = window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, highlight) {
      stream = await createDom();
      labelElement.innerText = label;
      labelElement.style.color = (highlight ? 'green' : 'orange');
      return await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
    }
    ''')

  display(js)
  
def take_photo(label, highlight):
  data = eval_js('takePhoto("{}", {})'.format(label, str(highlight).lower()))
  return data

label_map = imagenet.create_readable_names_for_imagenet_labels()
with tf.Session() as sess:
  saver.restore(sess,  checkpoint)

  start_input()  
  label = 'Capturing...'
  highlight = True
  while True:
    jpeg_b64 = take_photo(label, highlight)
    if not jpeg_b64:
      break

    jpeg_input = b64decode(jpeg_b64.split(',')[1])
    x = endpoints['Predictions'].eval(feed_dict={jpeg_input_tensor: jpeg_input})
    label = "Top prediction: {} (confidence: {:2.3f}%, id #{})".format(
        label_map[x.argmax()], x.max() * 100.0, x.argmax())
    highlight = x.max() >= 0.7
  

print('Finished')

INFO:tensorflow:Restoring parameters from mobilenet_v2_1.0_224.ckpt


<IPython.core.display.Javascript object>